In [3]:
#importing pandas and numpy libraries
import pandas as pd
import numpy as np
#Reading the dataset and previewing the first ten rows
Autolib_dataset = pd.read_csv('/content/Autolib_dataset (2).csv')
Autolib_dataset.head()

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23


### Data Cleaning

In [4]:
# Data cleaning action : Dropping Columns
# As we are only interested in the times of car rentals and their returning time, there are a number of columns that will not be put into use
# I decided to drop the column geopoint and scehduled.
# Geopoint is to be dropped as the station name, address and city columns provide us with enough information pertaining location.
# The schedule column details the dates that stations that have yet to be opened are to be opened. This column will therefore not be useful to us.
Autolib_dataset =Autolib_dataset.drop('Scheduled at', axis=1)
Autolib_dataset =Autolib_dataset.drop('Geo point', axis=1)


In [5]:
Autolib_dataset['Kind'].values

array(['STATION', 'STATION', 'STATION', ..., 'STATION', 'STATION',
       'STATION'], dtype=object)

In [21]:
# Data cleaning action: Checking for typos
# Upon data collection, a warning was given that there might be typos in the column charging status. As the has only a few possible values it is easy to check for this.
typos=[]
def typo_finder(x):
  for i in x.values:
    if i == 'nonexistent' or i == 'broken' or i == 'operational':
      pass
    else:
      typos.append(i)
      

typo_finder(Autolib_dataset['charging_status'])
print(typos)

[]


In [ ]:
Autolib_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Postal code          5000 non-null   int64 
 12  Public name          5000 non-null   object
 13  Rental status        5000 non-null   object
 14  Slots                5000 non-null   int64 
 15  Station type         5000 non-null   object
 16  Status

In [6]:
# Data Cleaning Action: Dropping irrelevant rows.
# Theere are a number of records in the dataset having their rental status as 'future' , their status as scheduled or their station type being sub-center.
# The columns mentioned and having their values as stated respectively, represent records of stations that have no resources as per time of recording.
# Meaning no cars could be rented from the station at the time and hence the records are not useful to us.

Autolib_dataset =Autolib_dataset[~((Autolib_dataset['Rental status'] == 'future') | (Autolib_dataset['Station type']== 'subs_center') | (Autolib_dataset['Status']=='scheduled') | (Autolib_dataset['Kind']== 'CENTRE'))]
Autolib_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4950 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              4950 non-null   object
 1   Cars                 4950 non-null   int64 
 2   Bluecar counter      4950 non-null   int64 
 3   Utilib counter       4950 non-null   int64 
 4   Utilib 1.4 counter   4950 non-null   int64 
 5   Charge Slots         4950 non-null   int64 
 6   Charging Status      4950 non-null   object
 7   City                 4950 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   4950 non-null   object
 10  Kind                 4950 non-null   object
 11  Postal code          4950 non-null   int64 
 12  Public name          4950 non-null   object
 13  Rental status        4950 non-null   object
 14  Slots                4950 non-null   int64 
 15  Station type         4950 non-null   object
 16  Status

In [11]:
# Data cleaning action: Checking accuracy of counters
# The car counter columns have a specified threshold and data exceeding these thresholds are inaccurate.
not_in_range =[]
def check(col):
  for i in Autolib_dataset[col]:

    if i > 7 :
      not_in_range.append(i)
    else:
      pass
    return not_in_range
  
check('bluecar_counter')

[]

In [12]:
Autolib_dataset[Autolib_dataset['utilib_counter'] > 4]

,address,bluecar_counter,utilib_counter,utilib_1.4_counter,charge_slots,charging_status,city,id,kind,postal_code,public_name,rental_status,slots,station_type,status,subscription_status,year,month,day,hour,minute


In [13]:
Autolib_dataset[Autolib_dataset['utilib_1.4_counter']> 5]

,address,bluecar_counter,utilib_counter,utilib_1.4_counter,charge_slots,charging_status,city,id,kind,postal_code,public_name,rental_status,slots,station_type,status,subscription_status,year,month,day,hour,minute


In [14]:
Autolib_dataset[Autolib_dataset['charge_slots'] > 3]

,address,bluecar_counter,utilib_counter,utilib_1.4_counter,charge_slots,charging_status,city,id,kind,postal_code,public_name,rental_status,slots,station_type,status,subscription_status,year,month,day,hour,minute


In [10]:
# Data Cleaning Action: Checking for missing values.
# We first identify the number of missing values in each column.
Autolib_dataset.isnull().any()
#The displayed comment column is the only column with missing values. The column will not necessarily be useful in our analysis.
# We can now drop the coulmn entirely.
Autolib_dataset= Autolib_dataset.dropna(axis=1, how= 'any')
Autolib_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4950 entries, 0 to 4999
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   address              4950 non-null   object
 1   bluecar_counter      4950 non-null   int64 
 2   utilib_counter       4950 non-null   int64 
 3   utilib_1.4_counter   4950 non-null   int64 
 4   charge_slots         4950 non-null   int64 
 5   charging_status      4950 non-null   object
 6   city                 4950 non-null   object
 7   id                   4950 non-null   object
 8   kind                 4950 non-null   object
 9   postal_code          4950 non-null   int64 
 10  public_name          4950 non-null   object
 11  rental_status        4950 non-null   object
 12  slots                4950 non-null   int64 
 13  station_type         4950 non-null   object
 14  status               4950 non-null   object
 15  subscription_status  4950 non-null   object
 16  year  

In [7]:
# Data Cleaning Action: Dropping dupilcate column cars
# The column cars is identical to column blue car counter and hence can be dropped.

Autolib_dataset =Autolib_dataset.drop('Cars', axis=1)


In [8]:
# Data cleaning action: Checking for duplicates in the dataframe.

np.count_nonzero(Autolib_dataset.duplicated())

0

In [9]:
# Data Cleaning Action: Standardization
# I prefer all my column labels to be in lower case as they are easier to call in that way.

Autolib_dataset.columns =Autolib_dataset.columns.str.lower().str.replace(" ","_")
Autolib_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4950 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   address              4950 non-null   object
 1   bluecar_counter      4950 non-null   int64 
 2   utilib_counter       4950 non-null   int64 
 3   utilib_1.4_counter   4950 non-null   int64 
 4   charge_slots         4950 non-null   int64 
 5   charging_status      4950 non-null   object
 6   city                 4950 non-null   object
 7   displayed_comment    111 non-null    object
 8   id                   4950 non-null   object
 9   kind                 4950 non-null   object
 10  postal_code          4950 non-null   int64 
 11  public_name          4950 non-null   object
 12  rental_status        4950 non-null   object
 13  slots                4950 non-null   int64 
 14  station_type         4950 non-null   object
 15  status               4950 non-null   object
 16  subscr

In [ ]:
# Data Cleaning Action: Datatype Conversion
# The last five columns ie year, month, day, hour and minute need to be consolidated in new columns as date and time datatype
Autolib_dataset['timestamp'] = pd.to_datetime(Autolib_dataset.iloc[:, 16:21])
Autolib_dataset.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4950 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   address              4950 non-null   object        
 1   bluecar_counter      4950 non-null   int64         
 2   utilib_counter       4950 non-null   int64         
 3   utilib_1.4_counter   4950 non-null   int64         
 4   charge_slots         4950 non-null   int64         
 5   charging_status      4950 non-null   object        
 6   city                 4950 non-null   object        
 7   id                   4950 non-null   object        
 8   kind                 4950 non-null   object        
 9   postal_code          4950 non-null   int64         
 10  public_name          4950 non-null   object        
 11  rental_status        4950 non-null   object        
 12  slots                4950 non-null   int64         
 13  station_type         4950 non-nul

In [ ]:
Autolib_dataset = Autolib_dataset.drop('year', axis=1)
Autolib_dataset = Autolib_dataset.drop('month', axis=1) 
Autolib_dataset = Autolib_dataset.drop('minute', axis=1)


In [ ]:
Autolib_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4950 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   address              4950 non-null   object        
 1   bluecar_counter      4950 non-null   int64         
 2   utilib_counter       4950 non-null   int64         
 3   utilib_1.4_counter   4950 non-null   int64         
 4   charge_slots         4950 non-null   int64         
 5   charging_status      4950 non-null   object        
 6   city                 4950 non-null   object        
 7   id                   4950 non-null   object        
 8   kind                 4950 non-null   object        
 9   postal_code          4950 non-null   int64         
 10  public_name          4950 non-null   object        
 11  rental_status        4950 non-null   object        
 12  slots                4950 non-null   int64         
 13  station_type         4950 non-nul

### Data Analysis


In [ ]:
# When the car counters are zero, there still seems to be a timestamp record. I interpreted this to mean when the no. of available cars is zero, the vehicles are being returned.
# I separated the dataset we had into two ie returned and picked up.
Returned = Autolib_dataset[Autolib_dataset['bluecar_counter'] == 0]
Picked_up = Autolib_dataset[Autolib_dataset['bluecar_counter'] > 0]

In [ ]:
# Determining the most popular hour of day for picking up a shared electric car in the city of Paris.

#Creating a dataset with only Paris as the city.
paris = Picked_up[Picked_up['city']== 'Paris']

#From the dataset paris , picking the columns bluecar_counter , hour , day and timestamp.                      
bluecar =paris[['bluecar_counter', 'hour', 'timestamp']]
bluecar.groupby('hour').sum().sort_values(by= 'bluecar_counter', ascending= False)

#hour 21 which is 9:00 pm seems to be the hour where most parisians rented an electric car as it has the highest number of cars rented.


,bluecar_counter
hour,
21,268
6,260
12,250
9,250
3,242
5,235
8,234
10,232
7,225


In [ ]:
 # Determining the most popular hour for returning cars
 popular_returned = Returned[['bluecar_counter', 'hour']]
 count =popular_returned.groupby('hour').count()
 count.sort_values(by= 'bluecar_counter', ascending= False)

 # Hour 14 has the highest count of vehicles being returned. This time is equivalent to 2:00pm.

,bluecar_counter
hour,
14,74
21,72
19,67
16,66
20,66
3,63
7,63
9,62
23,62


In [42]:
# Which station is most popular
stations =Autolib_dataset[['public_name', 'address', 'bluecar_counter']]
stations.groupby('public_name').count().sort_values(by= 'bluecar_counter', ascending= False).head(1)

# The station with the most appearences in the records appears to be Paris/Porte de Montrouge/8

,address,bluecar_counter
public_name,,
Paris/Porte de Montrouge/8,13,13


In [48]:
# Which station is the most popular at the most popular picking hour
nine_pm =Autolib_dataset[Autolib_dataset['hour']== 21]
nine_pm[['public_name', 'bluecar']].groupby('public_name').count().sort_values(by= 'address', ascending= False)

# The station most popular at 9 pm was Paris/Tronchet/19

,address
public_name,
Paris/Tronchet/19,4
Paris/Voltaire/182,3
Alfortville/Europe/22,2
ChÃ¢tenay-Malabry/Jean JaurÃ¨s/280,2
Paris/Ãmile Reynaud/4,2
...,...
Paris/Bertin PoirÃ©e/14,1
Paris/Bois/1,1
Paris/Bosquet/41,1


In [51]:
# What postal code is most popular for picking up blue cars
postal_code =Autolib_dataset[['postal_code', 'bluecar_counter']]
postal_code.groupby('postal_code').sum().sort_values(by= 'bluecar_counter', ascending= False).head(1)

,bluecar_counter
postal_code,
75015,609


In [53]:
Autolib_dataset[(Autolib_dataset['postal_code']== 75015) & (Autolib_dataset['public_name']== 'Paris/Porte de Montrouge/8')]
# The most popular station postal code is not equal to the postal code with a larger number of subscribers frequently using it.

,address,bluecar_counter,utilib_counter,utilib_1.4_counter,charge_slots,charging_status,city,id,kind,postal_code,public_name,rental_status,slots,station_type,status,subscription_status,day,hour,timestamp


In [54]:
# What postal code is most popular for picking up blue cars at the most popular picking hour.
nine_postal =nine_pm[['postal_code', 'bluecar_counter']]
nine_postal.groupby('postal_code').sum().sort_values(by= 'bluecar_counter', ascending= False).head(1)

,bluecar_counter
postal_code,
75014,36


In [ ]:
#